In [1]:
%matplotlib inline

In [2]:
import numpy as np

import matplotlib.pyplot as plt

import pandas.io.sql as psql
import psycopg2 as pg

import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.engine.saving import model_from_json
from keras.callbacks import EarlyStopping

from sklearn import model_selection

from imblearn.over_sampling import SMOTE

/Users/ca/PycharmProjects/JupyterNoteBookExample/venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Using TensorFlow backend.


In [6]:
with pg.connect(database='chi-navi',
                host='localhost',
                user='postgres',
                port=5432) as conn:
    all_season_sql = \
        "with spot_bus_stop as( " \
        "select " \
        "bus_stop_id,count(s.spot_id) as spot_num " \
        "from " \
        "(select bus_stop_id,latlon::point as latlon from bus_stop) as p " \
        "left outer join " \
        "spot s " \
        "on " \
        "ST_Covers( " \
        "ST_Buffer( " \
        "ST_POINT(s.area[0], s.area[1])::geography,400), " \
        "ST_POINT(p.latlon[1], p.latlon[0]) " \
        ") " \
        "group by bus_stop_id " \
        "), " \
        "delay_info as( " \
        "with route_bus_stop_list as( " \
        "select " \
        "t.task_id,r.route_id,b.bus_stop_id,rbs.ordinal " \
        "from " \
        "bus_stop b,route r,route_bus_stop rbs,task t " \
        "where " \
        "b.bus_stop_id = rbs.bus_stop_id " \
        "and r.route_id = rbs.route_id " \
        "and t.route_id = r.route_id " \
        "order by r.route_id,rbs.ordinal " \
        "), " \
        "delay_list as( " \
        "with stddev as( " \
        "select " \
        "stddev_samp(extract(epoch from (collide_at - departure_time))) as stddev " \
        "from " \
        "task_delay " \
        "where " \
        "(collide_at - departure_time) > '00:00:00' " \
        "), " \
        "delay_avg as( " \
        "select " \
        "avg((extract(epoch from (collide_at - departure_time)))) as delay_avg " \
        "from " \
        "task_delay " \
        "where " \
        "(collide_at - departure_time) > '00:00:00' " \
        ") " \
        "select  " \
        "td.task_id,td.departure_time,td.bus_stop_id,td.submit_in,td.collide_at " \
        ",extract(epoch from (td.collide_at - td2.departure_time)) as delay  " \
        "from " \
        "task_delay td,delay_avg da,stddev std,task_detail td2 " \
        "where " \
        "(extract(epoch from (td.collide_at - td2.departure_time)) between " \
        "da.delay_avg - std.stddev * 3 and da.delay_avg + std.stddev * 3) " \
        "and extract(epoch from (td.collide_at - td2.departure_time)) > 0 " \
        "and td.task_id = td2.task_id " \
        "and td.bus_stop_id = td2.bus_stop_id " \
        "and td.departure_time = td2.departure_time " \
        ") " \
        "select " \
        "distinct r.route_id,r.bus_stop_id,r.ordinal,d.collide_at, " \
        "d.task_id,d.delay,d.submit_in,d.departure_time, " \
        "date_part('HOUR' , d.departure_time) as hour, date_part('minutes',d.departure_time) as minutes " \
        "from " \
        "delay_list d, route_bus_stop_list r " \
        "where " \
        "d.bus_stop_id = r.bus_stop_id " \
        "and r.task_id = d.task_id " \
        "), " \
        "access_info as( " \
        "select " \
        "count(distinct session_id) as access_num,browsing_at::date as date, " \
        "date_part('HOUR', browsing_at) as hour, " \
        "case when " \
        "extract(dow from browsing_at::date) = 0 or extract(dow from browsing_at::date) = 6 then 0 " \
        "else 1 " \
        "end as youbi " \
        "from " \
        "user_browsing_log ubl " \
        "group by date,hour " \
        ") " \
        "select " \
        "case " \
        "when delay between 0 and 120 then 0 " \
        "when delay between 121 and 240 then 1 " \
        "when delay >= 241 then 2 " \
        "end as delay, " \
        "case " \
        "when ai.hour in(7,8,9,17,18,19) and ai.youbi = 1 then 0 " \
        "when ai.hour in(9,10,11,12,1,2) and ai.youbi = 0 then 1 " \
        "when ai.hour in(17,18,19,20,21,22) and ai.youbi = 0 then 2 " \
        "when ai.hour in(20,21,22) and ai.youbi = 1 then 1 " \
        "when ai.hour in(5,6,7,8) and ai.youbi = 0 then 3 " \
        "when ai.hour in(5,6) and ai.youbi = 1 then 4 " \
        "when ai.hour in(23,24) then 5 " \
        "end as time, " \
        "di.minutes, " \
        "ai.access_num,ai.youbi, " \
        "case " \
        "when submit_in = '2017-07-23' then 1 /* 航空祭 */ " \
        "when (submit_in between '2018-01-26' and '2018-02-18') or " \
        "(submit_in between '2017-01-27' and '2017-02-19') " \
        "then 2/* ひょうとう祭り */ " \
        "when submit_in between '2017-07-06' and '2017-07-09' then 3/* セガサミーカップ */ " \
        "else 0 " \
        "end as event, " \
        "case " \
        "when di.route_id in (13,14,23,24,55,56,58,59,60,61,62,63,64,67,68,69,70,71,74 " \
        ",78,79,80,81,82,91,92,93,94,95,96,97,98,99,106,107,108,109,11) " \
        "then 1 " \
        "else 0 " \
        "end as destination, " \
        "case " \
        "	when date_part('month',submit_in) in (3,4,5,6,9,10,11) then 1 " \
        "	when date_part('month', submit_in) in (7,8) then 0 " \
        "	when date_part('month', submit_in) in (12,1,2) then 2 " \
        "end as season, " \
        "di.bus_stop_id, " \
        "s.spot_num " \
        "from " \
        "spot_bus_stop s, " \
        "delay_info di, " \
        "access_info ai " \
        "where " \
        "di.hour = ai.hour " \
        "and di.submit_in = ai.date " \
        "and s.bus_stop_id = di.bus_stop_id " \
        "and delay <= 360 "

In [7]:
sql_result = psql.read_sql(all_season_sql, conn)

In [8]:
df = sql_result.groupby('bus_stop_id', as_index=False)

bus_stop_id_list = list(df.apply(lambda d: d['bus_stop_id'].drop_duplicates()))

In [14]:
def bus_stop_data_selection(bus_stop_id):
    filtered = sql_result[sql_result['bus_stop_id'] == bus_stop_id]
    label_two = filtered[filtered['delay'] == 2]
    label_zero = filtered[filtered['delay'] == 0]
    label_one = filtered[filtered['delay'] == 1]
    length = 0
  
    if len(label_zero) == 0 or len(label_one) == 0 or len(label_two) == 0:
        print('どれかのラベルが1つもありません')
    elif len(label_zero) <= len(label_one) and len(label_zero) <= len(label_two):
        label_one = label_one.sample(len(label_zero))
        label_two = label_two.sample(len(label_zero))
        length = len(label_zero)
    elif len(label_one) <= len(label_zero) and len(label_one) <= len(label_two):
        label_zero = label_one.sample(len(label_one))
        label_two = label_two.sample(len(label_one))
        length = len(label_one)
    else:
        label_zero = label_zero.sample(len(label_two))
        label_one = label_one.sample(len(label_two))
        length = len(label_two)

    x = pd.concat([label_zero, label_one, label_two], axis=0)
    
    y = x['delay']
    x = x[['time', 'minutes', 'youbi', 'destination', 'event', 'spot_num', 'season']]
    
    x_train, x_test, y_train, y_test = model_selection.train_test_split(
            x, y, train_size=int(len(x) * 0.9))

    y_train = np.identity(3)[y_train.astype(int)]
    
    return x_train, y_train, x_test, y_test, length


In [15]:
def data_selection():
    label_two = sql_result[sql_result['delay'] == 2]
    label_zero = sql_result[sql_result['delay'] == 0]
    label_one = sql_result[sql_result['delay'] == 1]
    length = 0
        
    if len(label_zero) == 0 or len(label_one) == 0 or len(label_two) == 0:
        print('どれかのラベルが1つもありません')
    elif len(label_zero) <= len(label_one) and len(label_zero) <= len(label_two):
        label_one = label_one.sample(len(label_zero))
        label_two = label_two.sample(len(label_zero))
        length = len(label_zero)
    elif len(label_one) <= len(label_zero) and len(label_one) <= len(label_two):
        label_zero = label_one.sample(len(label_one))
        label_two = label_two.sample(len(label_one))
        length = len(label_one)
    else:
        label_zero = label_zero.sample(len(label_two))
        label_one = label_one.sample(len(label_two))
        length = len(label_two)
        
    x = pd.concat([label_zero, label_one, label_two], axis=0)
    y = x['delay']
    x = x[['time', 'minutes', 'youbi', 'destination', 'event', 'spot_num', 'season']]
    
    x_train, x_test, y_train, y_test = model_selection.train_test_split(
            x, y, train_size=int(len(x) * 0.9))

    y_train = np.identity(3)[y_train.astype(int)]

    return x_train, y_train, x_test, y_test, length


In [24]:
from keras.layers import Dropout, BatchNormalization
from keras.optimizers import Adam

model = Sequential()

#入力層
model.add(Dense(units=3, input_dim=7, activation='relu', init='he_normal'))
model.add(BatchNormalization())

#中間層1
model.add(Dense(units=10, activation='relu', init='he_normal'))
model.add(BatchNormalization())

#中間層2
model.add(Dense(units=7, activation='relu', init='he_normal'))
model.add(BatchNormalization())

#中間層3
model.add(Dense(units=4, activation='relu', init='he_normal'))
model.add(BatchNormalization())

#出力層
model.add(Dense(units=3, activation='softmax', init='he_normal'))

model.compile(
    optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/Users/ca/PycharmProjects/JupyterNoteBookExample/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3, input_dim=7, activation="relu", kernel_initializer="he_normal")`
  import sys
/Users/ca/PycharmProjects/JupyterNoteBookExample/venv/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, activation="relu", kernel_initializer="he_normal")`
  # This is added back by InteractiveShellApp.init_path()


/Users/ca/PycharmProjects/JupyterNoteBookExample/venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, activation="relu", kernel_initializer="he_normal")`
  from ipykernel import kernelapp as app


/Users/ca/PycharmProjects/JupyterNoteBookExample/venv/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=4, activation="relu", kernel_initializer="he_normal")`


/Users/ca/PycharmProjects/JupyterNoteBookExample/venv/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3, activation="softmax", kernel_initializer="he_normal")`


In [32]:
# for bus_stop_id in enumerate(bus_stop_id_list):
# 
#     x_train, y_train, X_test, Y_test, length = bus_stop_data_selection(bus_stop_id[1])

x_train, y_train, X_test, Y_test, length = data_selection()
if length != 0:
    batch_size = int(len(x_train) * 0.2)
    epochs = 100

    # print('------------------------ bus_stop_id {} ------------------------\n'.format(bus_stop_id[1]))

    hist = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                     callbacks=[EarlyStopping(monitor='loss', patience=10, verbose=1)])


Epoch 1/100


176331/881655 [=====>........................] - ETA: 1s - loss: nan - acc: 0.3339

352662/881655 [===========>..................] - ETA: 1s - loss: nan - acc: 0.3338

528993/881655 [=================>............] - ETA: 0s - loss: nan - acc: 0.3338

705324/881655 [=======================>......] - ETA: 0s - loss: nan - acc: 0.3336

...

881655/881655 [==============================] - 2s 2us/step - loss: nan - acc: 0.3334


Epoch 7/100


176331/881655 [=====>........................] - ETA: 1s - loss: nan - acc: 0.3349

352662/881655 [===========>..................] - ETA: 1s - loss: nan - acc: 0.3340

KeyboardInterrupt: 

In [26]:
Y_test = np.identity(3)[Y_test.astype(int)]
score = model.evaluate(X_test, Y_test)
print(score)

   32/97962 [..............................] - ETA: 18:45

  928/97962 [..............................] - ETA: 43s  

 1632/97962 [..............................] - ETA: 27s

 2816/97962 [..............................] - ETA: 17s

 3936/97962 [>.............................] - ETA: 13s

...

94112/97962 [===========================>..] - ETA: 0s

95520/97962 [============================>.] - ETA: 0s

96960/97962 [============================>.] - ETA: 0s

97962/97962 [==============================] - 5s 47us/step


[nan, 0.3337212388480207]
